In [1]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [2]:
#Data Latih
df = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')

corpusA = df.sms.tolist()
labelA = df.label.tolist()
len(df)

1143

# Ekstraksi Fitur

In [3]:
fitur = lib.feature_extraction(corpusA)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [4]:
jumlah_populasi = 200
jumlah_point = 1000 #jumlah titik point crossover
prob_mutasi = 0.2 #probabilitas permutasi

K=25

metode = "tfidf"

alpha = 1
target = 0.99 #target akurasi

panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
# temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)

print(len(fitnes_pop),"|",len(pop))

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

best_jumlah_fitur_list = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_titik = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
        a3 = anak_binary[2]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1
        if sum(a3)<=0:
            a3[-1]=1
            a3[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])
            
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})
            
        str_a3 = lib.int_to_str(a3)
        if str_a3 in pop_used:
            family_fitur.append(a3)
            family_fitness.append(pop_used[str_a3])
        else:
            family_fitur.append(a3)
            fitness_anak_3 = lib.fitness_kf(corpusA, labelA, a3, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_3)
            pop_used.update({str_a3:fitness_anak_3})

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)
        best_jumlah_fitur_list.append(sum(best_family_fitur))

#         print("   ",best_family_fitness)
        print("   ",lib.tampil_finess(best_family_fitness),"|",str(sum(best_family_fitur))+"|"+str(len(best_family_fitur)),"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

fitness all features: 0.9342995169082127
200 | 200
-------------------------
Generasi ke 1 [Terbaik]
    0.9247342995169083   | 3288|4378 | 1
    0.9273623188405798   | 3276|4378 | 1
    0.9151304347826088   | 3281|4378 | 1
    0.922144927536232    | 3313|4378 | 1
    0.9291400966183575   | 3283|4378 | 1
    0.9326376811594203   | 3271|4378 | 1
    0.9204444444444445   | 3279|4378 | 1
    0.9202898550724637   | 3330|4378 | 1
    0.9281739130434784   | 3308|4378 | 1
    0.9325990338164253   | 3287|4378 | 1
    0.9343574879227055   | 3279|4378 | 1
    0.9212753623188407   | 3331|4378 | 1
    0.930048309178744    | 3277|4378 | 1
    0.9256425120772949   | 3290|4378 | 1
    0.9291400966183576   | 3300|4378 | 1
    0.9203671497584541   | 3261|4378 | 1
    0.924695652173913    | 3229|4378 | 1
    0.9185700483091788   | 3251|4378 | 1
    0.9221835748792272   | 3311|4378 | 1
    0.9281739130434784   | 3267|4378 | 1
    0.9343188405797102   | 3250|4378 | 1
    0.9308599033816425   | 3266|4378 |

    0.937816425120773    | 4337|4378 | 6
-------------------------
Generasi ke 7 [Terbaik]
    0.9387246376811595   | 4111|4378 | 7
    0.937816425120773    | 4343|4378 | 7
-------------------------
Generasi ke 8 [Terbaik]
    0.9387246376811595   | 4111|4378 | 8
Best 0.9387246376811595
jumlah fitur      4111
jumlah fitur asli 4378
good_fitur


In [5]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_kf(corpusA, labelA, hasil_seleksi_fitur, fitur, alpha = alpha, metode = metode, K=K)

0.9387246376811595

In [6]:
# for i in np.array(fitur)[lib.get_index(hasil_seleksi_fitur)]:
#     print(i, end=", ")

In [16]:
import json
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

dict_dasddas = {
    "score":max(best_fitness_list),
    "fitur":fitur_pilihan.tolist(),
    "K":K
}

with open("fitur/fitur_p3.json","w") as f:
    json.dump(dict_dasddas,f)

4111


In [8]:

dict_data = {
    "fitness":best_fitness_list,
    "jumlah_fitur":best_jumlah_fitur_list,
    "generasi":best_generasi_list,
}

df_hasil = pd.DataFrame.from_dict(dict_data)
df_hasil


,fitness,jumlah_fitur,generasi
0,0.924734,3288,1
1,0.927362,3276,1
2,0.915130,3281,1
3,0.922145,3313,1
4,0.929140,3283,1
...,...,...,...
197,0.937816,4343,6
198,0.937816,4337,6
199,0.938725,4111,7
200,0.937816,4343,7


In [9]:
print(len(best_fitness_list))

202


In [10]:
cek = 0.93

grouped = df_hasil.groupby('generasi')
grouped.filter(lambda x: x['fitness'].mean() > cek)

,fitness,jumlah_fitur,generasi
100,0.930821,4096,2
101,0.930821,4100,2
102,0.930821,4120,2
103,0.928155,4128,2
104,0.934357,4093,2
...,...,...,...
197,0.937816,4343,6
198,0.937816,4337,6
199,0.938725,4111,7
200,0.937816,4343,7
